In [1]:
import pandas as pd
import numpy as np

In [1]:
import h5py
with h5py.File('train2.h5','r') as ipt:
    xtrain = ipt['x'][:]
    ytrain = ipt['y'][:]
    ytrainreal = ipt['yreal']
    xtest = ipt['test'][:]
    ytestreal = ipt['ytestreal'][:]

In [2]:
import torch
from torch import nn
import torch.optim as optim
import torch.utils.data as Data
import torch.nn.functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


NameError: name 'x_train' is not defined

In [3]:
plen = 200
tlen = 200
#x_train = torch.from_numpy(xtrain).view(-1,1,tlen,plen*4).cuda()
#y_train = torch.from_numpy(ytrain).view(-1).cuda()
x_test = torch.from_numpy(xtest).view(-1,1,tlen,plen*4).contiguous().cuda()

In [4]:
batch_size=64
train_dataset = Data.TensorDataset(x_train[:7000],y_train[:7000])
train_loader = Data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=False)
test_dataset = Data.TensorDataset(x_train[7000:],y_train[7000:])
test_loader = Data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

NameError: name 'x_train' is not defined

In [5]:
class Net(torch.nn.Module):
    def __init__(self, time_size=200,input_size=800,out_size=200):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,32,5)#(200-5+1=196,800-5+1=796)
        self.conv2 = nn.Conv2d(32,32,5)#(196-5+1=192,796-5+1=792)
        self.conv3 = nn.Conv2d(32,32,5)#(96-5+1=92,396-5+1=392)
        self.conv4 = nn.Conv2d(32,32,5)#(92-5+1=88,392-5+1=388)
        self.conv5 = nn.Conv2d(32,32,5)#(44-5+1=40,194-5+1=190)
        
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(32)
        self.bn4 = nn.BatchNorm2d(32)
        self.bn5 = nn.BatchNorm1d(512)
        self.bn6 = nn.BatchNorm2d(32)
        
        self.fc1 = nn.Linear(32*20*95, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(512,out_size)
        
        self.dropout = nn.Dropout(p=0.5)
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))# 196
        x = F.max_pool2d(F.relu(self.bn2(self.conv2(x))),(2,2))# (192/2=96,792/2=396)
        x = F.relu(self.bn3(self.conv3(x)))# 92,392
        x = F.max_pool2d(F.relu(self.bn4(self.conv4(x))),(2,2))# (88/2=44,388/2=194)
        #x = F.max_pool2d(self.bn4(x),(2,2))#(10/2=5)
        x = F.max_pool2d(F.relu(self.bn6(self.conv5(x))),(2,2))# (20,95)
        x = torch.flatten(x,1)
        x = self.dropout(F.relu(self.bn5(self.fc1(x))))
        #x = F.relu(self.fc3(x))
        x = self.fc2(x)
        return x
model = Net().to(device)

In [6]:
model.load_state_dict(torch.load('itemConv1.pth'))

<All keys matched successfully>

In [7]:
learning_rate=0.01
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#criterion = nn.MSELoss()
criterion=nn.CrossEntropyLoss()

In [10]:
print(1)

1


In [11]:
epochs = 64
errrate = 0
for epoch in range(epochs):
    model.train()
    for batch,(x,y) in enumerate(train_loader):
        #print(x.shape)
        #print(y.shape)
        ypred = model(x.float())
        loss = criterion(ypred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    size = len(test_loader.dataset)
    test_loss, correct = 0,0
    with torch.no_grad():
        for x,y in test_loader:
            x = x.float()
            pred = model(x)
            test_loss += criterion(pred,y).item()
            correct += (pred.argmax(1) ==y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    if correct>errrate:
        errrate = correct
        torch.save(model.state_dict(),'itemConv1.pth')
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss:{test_loss:>8f} \n')

Test Error: 
 Accuracy: 0.0%, Avg loss:25.572383 



KeyboardInterrupt: 

In [13]:
torch.cuda.empty_cache()

In [ ]:
print(errrate)

In [ ]:
torch.save(model.state_dict(),'itemConv1.pth')

In [14]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 1            |        cudaMalloc retries: 1         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   33807 MB |   33885 MB |   39174 MB |    5367 MB |
|       from large pool |   33805 MB |   33883 MB |   39172 MB |    5366 MB |
|       from small pool |       2 MB |       2 MB |       2 MB |       0 MB |
|---------------------------------------------------------------------------|
| Active memory         |   33807 MB |   33885 MB |   39174 MB |    5367 MB |
|       from large pool |   33805 MB |   33883 MB |   39172 MB |

In [20]:
model.eval()
realpred = np.zeros((x_test.shape[0]),dtype=int)
with torch.no_grad():
    for i in range(x_test.shape[0]):
        x_t = x_test[i,:,:].view(-1,1,tlen,plen*4).cuda().float()
        predO = model(x_t)
        predO=predO[-1,:]
        pred = np.argmax(predO.cpu().numpy(),axis=-1)
        realpred[i] = ytestreal[i,pred]

In [11]:
import pandas as pd,numpy as np

In [21]:
sampleSubmit = pd.read_csv('./submit_example.csv')

In [22]:
realpredict = pd.DataFrame({'user_id':sampleSubmit['user_id'],'product_id':np.array(realpred).reshape(-1)})

In [23]:
realpredict.to_csv('itemTrainFilterC.csv',index=False)